<a href="https://colab.research.google.com/github/GonorAndres/Analisis_Numerico_2025-2/blob/main/Practica3/Ejercicio_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Ejercicio 25 – Comparación de métodos en un sistema complicado

#### a) ¿Cuál método se ve más afectado por el pequeño cambio en los datos?

Definitivamente el método de **ecuaciones normales con Cholesky** fue el más afectado. Aunque la perturbación que le metimos al vector $ \bar{y} $ fue pequeñísima ($ \varepsilon = 10^{-10} $), el resultado que dio para $ \tilde{x} $ se alejó bastante del vector original que sabíamos que era todo unos.

En cambio, el método de **QR** fue mucho más estable: dio una respuesta prácticamente idéntica a la verdadera.

#### b) ¿Cuál se acerca más al resultado exacto ($ x_i = 1 $)?

Sin duda, **QR**. El error entre la solución que nos dio y el vector exacto fue pequeñísimo (del orden de $ 10^{-4} $). Con Cholesky, el error fue grande (más de 3), lo cual nos dice que en este tipo de problemas puede fallar bastante.

#### c) ¿Esa diferencia en las soluciones afecta el ajuste a los datos? ¿Por qué?

Sí afecta, aunque no tanto como uno pensaría. Ambas soluciones logran ajustarse a los datos con cierto nivel de precisión, pero:

- El ajuste con **QR** fue mucho más exacto (error como de $ 10^{-10} $),
- mientras que el ajuste con **Cholesky** fue menos fino (error de $ 10^{-7} $).

Esto confirma que QR no solo se acerca más al valor exacto, sino que además logra un mejor ajuste a los datos que se le dieron, incluso si están un poco ruidosos.



In [5]:
# Reimportar y preparar lo necesario tras el reset
import numpy as np
import pandas as pd
from numpy.linalg import norm

# Volvemos a definir los datos del ejercicio 25 para aplicarlo con la rutina de Cholesky de clase
m = 21
n = 12
epsilon = 1e-10
t = np.linspace(0, 1, m)
A_25 = np.vander(t, N=n, increasing=True)
x_exacto = np.ones(n)
y_exactos = A_25 @ x_exacto

np.random.seed(0)
u = np.random.rand(m)
perturbacion = (2 * u - 1) * epsilon
y_perturbado = y_exactos + perturbacion

# Rutina Cholesky de la clase
def Cholesky(A):
    n = A.shape[0]
    L = np.zeros_like(A)

    for i in range(n):
        for j in range(i+1):
            suma = sum(L[i][k] * L[j][k] for k in range(j))
            if i == j:
                L[i][j] = np.sqrt(A[i][i] - suma)
            else:
                L[i][j] = (A[i][j] - suma) / L[j][j]
    return L

def Solver_Cholesky(A, b):
    L = Cholesky(A)
    y = SustitucionDelante(L, b)
    x = SustitucionAtras(L.T, y)
    return x

# Resolver con la rutina de clase
AtA_clase = A_25.T @ A_25
Atb_clase = A_25.T @ y_perturbado
x_cholesky_clase = Solver_Cholesky(AtA_clase, Atb_clase)

# Resolver con QR para comparar
Q, R = np.linalg.qr(A_25)
x_qr_clase = np.linalg.solve(R, Q.T @ y_perturbado)

# Comparación
error_cholesky = norm(x_cholesky_clase - x_exacto)
error_qr = norm(x_qr_clase - x_exacto)

# Ajuste del polinomio
y_fit_cholesky = A_25 @ x_cholesky_clase
y_fit_qr = A_25 @ x_qr_clase

# Error de ajuste
ajuste_cholesky = norm(y_fit_cholesky - y_perturbado)
ajuste_qr = norm(y_fit_qr - y_perturbado)

# Tabla final
df_25_clase = pd.DataFrame({
    "Método": ["Ecuaciones normales (Cholesky, clase)", "QR"],
    "Error respecto a x exacto": [error_cholesky, error_qr],
    "Error de ajuste ||Ax - y||": [ajuste_cholesky, ajuste_qr]
})

df_25_clase


,Método,Error respecto a x exacto,Error de ajuste ||Ax - y||
0,"Ecuaciones normales (Cholesky, clase)",0.884261,3.809340e-08
1,QR,0.000124,1.563777e-10


In [2]:
#Funciones de la clase


import numpy as np

def SubMat(Mat, ren, col):
    """
    Crea una submatriz eliminando un renglón y una columna específicos de la matriz original.

    Parámetros:
    -----------
    Mat : numpy.ndarray
        Matriz original de la cual se obtendrá la submatriz.
    ren : int
        Índice del renglón que se eliminará de la matriz.
    col : int
        Índice de la columna que se eliminará de la matriz.

    Retorna:
    --------
    numpy.ndarray
        Submatriz resultante después de eliminar el renglón y la columna especificados.

    Ejemplo:
    --------
    >>> Mat = np.array([[1, 2, 3],
    ...                [4, 5, 6],
    ...                [7, 8, 9]])
    >>> SubMat(Mat, 1, 1)
    array([[1, 3],
           [7, 9]])
    """
    # Crear una copia de la matriz original para no modificarla
    M1 = np.copy(Mat)

    # Eliminar el renglón especificado
    M1 = np.delete(M1, ren, axis=0)

    # Eliminar la columna especificada
    M1 = np.delete(M1, col, axis=1)

    return M1


import numpy as np

def Det(Mat):
    """
    Calcula el determinante de una matriz cuadrada de manera recursiva.

    Parámetros:
    -----------
    Mat : numpy.ndarray
        Matriz cuadrada de la cual se calculará el determinante.
        Debe ser de tamaño n x n, donde n >= 2.

    Retorna:
    --------
    float
        El determinante de la matriz.

    Ejemplo:
    --------
    >>> Mat = np.array([[1, 2],
    ...                [3, 4]])
    >>> Det(Mat)
    -2.0

    >>> Mat = np.array([[6, 1, 1],
    ...                [4, -2, 5],
    ...                [2, 8, 7]])
    >>> Det(Mat)
    -306.0
    """
    # Caso base: matriz 2x2
    if Mat.shape[0] == 2 and Mat.shape[1] == 2:
        return Mat[0][0] * Mat[1][1] - (Mat[0][1] * Mat[1][0])

    # Caso recursivo: matrices más grandes
    deter = 0.0
    for col in range(Mat.shape[0]):
        # Calcula el cofactor y suma al determinante
        deter += ((-1) ** col) * Mat[0][col] * Det(SubMat(Mat, 0, col))
    return deter


import numpy as np

def Transpuesta(Mat):
    """
    Calcula la transpuesta de una matriz cuadrada modificando la matriz original.

    Parámetros:
    -----------
    Mat : numpy.ndarray
        Matriz cuadrada de tamaño n x n que se transpondrá.
        La matriz se modificará in situ.

    Retorna:
    --------
    numpy.ndarray
        La matriz transpuesta. La matriz original también se modifica.

    Ejemplo:
    --------
    >>> Mat = np.array([[1, 2, 3],
    ...                [4, 5, 6],
    ...                [7, 8, 9]])
    >>> Transpuesta(Mat)
    array([[1, 4, 7],
           [2, 5, 8],
           [3, 6, 9]])
    """
    for ren in range(Mat.shape[0]):
        for col in range(Mat.shape[1]):
            if ren < col:
                # Intercambia los elementos para obtener la transpuesta
                Mat[ren, col], Mat[col, ren] = Mat[col, ren], Mat[ren, col]
    return Mat



import numpy as np

def Cofactores(Mat):
    """
    Calcula la matriz de cofactores de una matriz cuadrada.

    Parámetros:
    -----------
    Mat : numpy.ndarray
        Matriz cuadrada de tamaño n x n para la cual se calcularán los cofactores.

    Retorna:
    --------
    numpy.ndarray
        Matriz de cofactores, donde cada elemento es el cofactor correspondiente
        de la matriz original.

    Ejemplo:
    --------
    >>> Mat = np.array([[1, 2],
    ...                [3, 4]])
    >>> Cofactores(Mat)
    array([[ 4., -3.],
           [-2.,  1.]])
    """
    # Crear una matriz de ceros del mismo tamaño que Mat para almacenar los cofactores
    Cofa = np.zeros_like(Mat, dtype=float)

    # Calcular el cofactor para cada elemento de la matriz
    for ren in range(Mat.shape[0]):
        for col in range(Mat.shape[1]):
            # Calcular el determinante de la submatriz (menor) y aplicar el signo
            Cofa[ren, col] = ((-1) ** (ren + col)) * Det(SubMat(Mat, ren, col))
    return Cofa



import numpy as np

def Inv(Mat):
    """
    Calcula la inversa de una matriz cuadrada utilizando la matriz de cofactores.

    Parámetros:
    -----------
    Mat : numpy.ndarray
        Matriz cuadrada de tamaño n x n que se invertirá.
        Debe ser una matriz no singular (su determinante debe ser distinto de cero).

    Retorna:
    --------
    numpy.ndarray
        La matriz inversa de la matriz original.

    Ejemplo:
    --------
    >>> Mat = np.array([[4, 7],
    ...                [2, 6]])
    >>> Inv(Mat)
    array([[ 0.6, -0.7],
           [-0.2,  0.4]])
    """
    # Calcular el determinante de la matriz
    deter = Det(Mat)

    # Verificar si la matriz es singular (determinante = 0)
    if deter == 0:
        raise ValueError("La matriz es singular y no tiene inversa.")

    # Calcular la matriz de cofactores
    Cofac = Cofactores(Mat)

    # Transponer la matriz de cofactores para obtener la matriz adjunta
    Cofac = Transpuesta(Cofac)

    # Calcular la inversa multiplicando la adjunta por 1/determinante
    Inversa = (1 / deter) * Cofac

    return Inversa



A=np.array([[2,3,-4],[0,-4,2],[1,-1,5]])
b=np.array([1.0,1.0,1.0])

import numpy as np

def SolveInv(Mat, vec):
    """
    Resuelve un sistema de ecuaciones lineales utilizando la matriz inversa.

    Parámetros:
    -----------
    Mat : numpy.ndarray
        Matriz cuadrada de coeficientes del sistema de ecuaciones.
        Debe ser una matriz no singular (su determinante debe ser distinto de cero).

    vec : numpy.ndarray
        Vector de términos independientes.
        Su tamaño debe coincidir con el número de filas de la matriz `Mat`.

    Retorna:
    --------
    numpy.ndarray
        Vector solución del sistema de ecuaciones `Mat @ x = vec`.

    Excepciones:
    ------------
    ValueError
        Se lanza un error si la matriz `Mat` no es invertible.

    Ejemplo:
    --------
    >>> Mat = np.array([[2, 1],
    ...                 [5, 3]])
    >>> vec = np.array([4, 10])
    >>> SolveInv(Mat, vec)
    array([2., 0.])

    """
    # Calcular la inversa de la matriz
    InvMat = Inv(Mat)

    # Multiplicar la inversa por el vector de términos independientes
    Solucion = InvMat @ vec

    return Solucion


import numpy as np
from numpy import linalg as LA


def SustitucionDelante(Mat, b):
    """
    Realiza la sustitución hacia adelante para resolver un sistema de ecuaciones lineales
    representado por una matriz triangular inferior.

    Parámetros:
    -----------
    Mat : numpy.ndarray
        Una matriz triangular inferior de tamaño n x n.
    b : numpy.ndarray
        Un vector de términos independientes de tamaño n.

    Retorna:
    --------
    x : numpy.ndarray
        Un vector solución de tamaño n que satisface la ecuación Mat @ x = b.

    Descripción:
    ------------
    Esta función resuelve un sistema de ecuaciones lineales de la forma Mat @ x = b,
    donde Mat es una matriz triangular inferior. Utiliza el método de sustitución hacia adelante,
    comenzando desde la primera fila de la matriz y avanzando hacia la última.

    Ejemplo:
    --------
    >>> Mat = np.array([[1, 0, 0],
    ...                 [2, 3, 0],
    ...                 [4, 5, 6]])
    >>> b = np.array([1, 8, 32])
    >>> SustitucionDelante(Mat, b)
    array([1., 2., 3.])
    """
    n = Mat.shape[0]
    x = np.zeros(n)

    for i in range(n):
        SumCum = 0.0
        for j in range(i):
            SumCum += Mat[i, j] * x[j]
        x[i] = (b[i] - SumCum) / Mat[i, i]

    return x

def SustitucionAtras(Mat, b):
    """
    Realiza la sustitución hacia atrás para resolver un sistema de ecuaciones lineales
    representado por una matriz triangular superior.

    Parámetros:
    -----------
    Mat : numpy.ndarray
        Una matriz triangular superior de tamaño n x n.
    b : numpy.ndarray
        Un vector de términos  independientes de tamaño n.

    Retorna:
    --------
    x : numpy.ndarray
        Un vector solución de tamaño n que satisface la ecuación Mat @ x = b.

    Descripción:
    ------------
    Esta función resuelve un sistema de ecuaciones lineales de la forma Mat @ x = b,
    donde Mat es una matriz triangular superior. Utiliza el método de sustitución hacia atrás,
    comenzando desde la última fila de la matriz y avanzando hacia la primera.

    Ejemplo:
    --------
    >>> Mat = np.array([[3, 2, 1],
    ...                 [0, 2, 1],
    ...                 [0, 0, 1]])
    >>> b = np.array([6, 4, 1])
    >>> SustitucionAtras(Mat, b)
    array([1., 1., 1.])
    """
    n = Mat.shape[0]
    x = np.zeros(n)

    for i in range(n-1, -1, -1):
        SumCum = 0.0
        for j in range(i+1, n):
            SumCum += Mat[i, j] * x[j]
        x[i] = (b[i] - SumCum) / Mat[i, i]

    return x

def LU(A):
  U=np.copy(A)
  L=np.eye(A.shape[0])

  for i in range(A.shape[0]):
    Li=np.eye(A.shape[0])
    for j in range(i+1,A.shape[0]):
      Li[j,i]=-U[j,i]/U[i,i]
      L[j,i]=U[j,i]/U[i,i]
    U=Li@U
  return L,U

def Solver_LU(A,b):
  L,U=LU(A)
  # El sistema que se resuelve es Ly=b
  y=SustitucionDelante(L, b)
  # El sistema que se resuelve es Ux=y
  x=SustitucionAtras(U, y)

  return x

## Funciones de ejercicios anteriores de la tarea
def LU_parcial(A):
    n = A.shape[0]
    U = np.copy(A)
    L = np.eye(n)
    P = np.eye(n)

    for i in range(n):
        # Encontrar fila con valor absoluto máximo en la columna i
        max_val = abs(U[i, i])
        max_row = i
        for k in range(i+1, n):
            if abs(U[k, i]) > max_val:
                max_val = abs(U[k, i])
                max_row = k

        # Intercambiar filas si es necesario
        if max_row != i:
            U[[i, max_row], :] = U[[max_row, i], :] # el corchete dentro de la posición de las filas sirve como multiasignación así no creamos variables de apoyo
            P[[i, max_row], :] = P[[max_row, i], :]
            if i > 0: # tenemos que solo se intercambia después de la primera interación por justamente en la primer vuelta no hemos calculado nada y sigue siendo la identidad solo intercambiariamos ceros identicos
                L[[i, max_row], :i] = L[[max_row, i], :i] # sin embargo para la segunda vuelta ya habremos calculado por lo menos un elemento que no esta en la diagonal de la columna 1, entonces ya es necesario intercambiar
                                                          # y solo intercambiamos hasta antes de la columna i, pues solo nos interesan la parte triangular inferior, la diagonal son unos por definición
        # Eliminación gaussiana
        for j in range(i+1, n):
            L[j, i] = U[j, i] / U[i, i]
            U[j, :] = U[j, :] - (L[j, i] * U[i, :])

    return P, L, U

def Solver_LUParcial(A, b):
    # Resuelve el sistema Ax = b usando la factorización LU con pivoteo parcial.
    # como Ax =b y PA = LU entonces LUx = PAx = Pb así resolvemos
    # LUx = Pb
    P, L, U = LU_parcial(A)
    Pb = P @ b
    z = SustitucionDelante(L, Pb)
    x = SustitucionAtras(U, z)
    return x

def LU_total(A):
    n = A.shape[0]
    U = np.copy(A)
    L = np.eye(n)
    P = np.eye(n)
    Q = np.eye(n)

    for i in range(n):
        # Buscar el índice del valor absoluto máximo en submatriz U[i:, i:]
        max_val = abs(U[i, i])
        max_row, max_col = i, i
        for r in range(i, n):
            for s in range(i, n):
                if abs(U[r, s]) > max_val:
                    max_val = abs(U[r, s])
                    max_row, max_col = r, s

        # Intercambiar filas si es necesario
        if max_row != i:
            U[[i, max_row], :] = U[[max_row, i], :] # el corchete dentro de la posición de las filas sirve como multiasignación así no creamos variables de apoyo
            P[[i, max_row], :] = P[[max_row, i], :]
            if i > 0: # tenemos que solo se intercambia después de la primera interación por justamente en la primer vuelta no hemos calculado nada y sigue siendo la identidad solo intercambiariamos ceros identicos
                L[[i, max_row], :i] = L[[max_row, i], :i] # sin embargo para la segunda vuelta ya habremos calculado por lo menos un elemento que no esta en la diagonal de la columna 1, entonces ya es necesario intercambiar
                                                          # y solo intercambiamos hasta antes de la columna i, pues solo nos interesan la parte triangular inferior, la diagonal son unos por definición


        # Intercambio de columnas si es necesario
        if max_col != i:
            U[:, [i, max_col]] = U[:, [max_col, i]]
            Q[:, [i, max_col]] = Q[:, [max_col, i]]
            # Ahora vemos que no intercambiamos las columnas de L en ninguna ocasión porque L guarda en la parte triangular inferior todos los pivotes correspondientes a cada fila para la eliminación de los terminos debajo de la diagonal
            # sin embargo ahora los renglones no se movieron y el pivote de eliminación sigue correspondiendo a esa misma fila, tampoco debemos tener cuidado en el cambio de orden de las variables
            # pues el pivote afecta a toda la fila por igual no importa en que orden se encuentre y el cambio en U no afecta en nada

        # Eliminación gaussiana
        for j in range(i+1, n):
            L[j, i] = U[j, i] / U[i, i]
            U[j, :] = U[j, :] - (L[j, i] * U[i, :])

    return P, L, U, Q

def Solver_LUtotal(A, b):
    # Resuelve el sistema Ax = b usando la factorización LU con pivoteo total.
    # como Ax = b y PAQ = LU entonces PA = LUQ^-1, LUQ^-1x = PAx = Pb entonces sea y = Q^-1x tenemos que LUy = Pb
    #  Resolvemos LUy = Pb
    P, L, U, Q = LU_total(A)
    Pb = P @ b
    z = SustitucionDelante(L, Pb)
    y = SustitucionAtras(U, z)
    # Resolvemos y = Q^-1x donde x = Qy
    x = Q @ y

    return x

def resolver_sistema_por_gauss(A, b):
    """Resuelve Ax = b usando eliminación de Gauss sin pivoteo."""
    A = A.astype(float)
    b = b.astype(float)
    n = len(b)

    for i in range(n):
        for j in range(i + 1, n):
            factor = A[j, i] / A[i, i]
            A[j, i:] -= factor * A[i, i:]
            b[j] -= factor * b[i]

    x = np.zeros(n)
    for i in reversed(range(n)):
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]

    return x

def minimos_cuadrados_manual(A, b):
    AtA = A.T @ A
    Atb = A.T @ b
    return resolver_sistema_por_gauss(AtA.copy(), Atb.copy())

def condicion_matriz_linalg(A):
    return cond(A, 2)